In [3]:
from ImageProcessor import ImageProcessor
import numpy as np
import cv2
import dlib

In [4]:
# Global initialization
global_detector = dlib.get_frontal_face_detector()
global_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
global_sr_model = cv2.dnn_superres.DnnSuperResImpl_create()
global_sr_model.readModel("EDSR_x4.pb")
global_sr_model.setModel("edsr", 4)
ImageProcessor = ImageProcessor(global_detector, global_predictor, global_sr_model)

In [5]:
def get_combined_eyes(frame, global_sr_model, global_detector, global_predictor):
    """
    
    Detects and combines the eye regions from the frame using the extract_eye_region method.
    Args:
        frame: The input image frame.
        landmarks: Facial landmarks detected in the frame.
        global_sr_model: Super-resolution model.
        global_detector: Face detector.
        global_predictor: Landmark predictor.
    Returns:
        The combined eye regions, or None if not detected.
    """
    # Enhance image resolution
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = global_detector(gray)

    cv2.imshow('frame', frame)
    cv2.waitKey(0)

    for face in faces:
        landmarks = global_predictor(gray, face)
        #draw the landmarks on the face
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # Extract the coordinates for each eye using dlib's facial landmark indices
        left_eye_points = range(36, 42)
        right_eye_points = range(42, 48)

        # Extract each eye region using the extract_eye_region method
        left_eye_region, _ = ImageProcessor.extract_eye_region(frame, landmarks, left_eye_points)
        right_eye_region, _ = ImageProcessor.extract_eye_region(frame, landmarks, right_eye_points)

        cv2.imshow('left_eye', left_eye_region)
        cv2.imshow('right_eye', right_eye_region)
        cv2.waitKey(0)


        left_eye_super_res = ImageProcessor.enhance_image_resolution(left_eye_region, global_sr_model)
        right_eye_super_res = ImageProcessor.enhance_image_resolution(right_eye_region, global_sr_model)
        cv2.imshow('left_eye_super_res', left_eye_super_res)
        cv2.imshow('right_eye_super_res', right_eye_super_res)
        cv2.waitKey(0)

        # Resize each eye region to the target size
        target_size = (40, 48)  # Assuming you want to resize both eyes to this size
        left_eye_resized = cv2.resize(left_eye_super_res, target_size).astype(np.uint8)/255

        right_eye_resized = cv2.resize(right_eye_super_res, target_size).astype(np.uint8)/255

        # Combine the eye regions
        combined_eyes = np.hstack((left_eye_resized, right_eye_resized))
        cv2.imshow('combined_eyes', combined_eyes)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return combined_eyes

    return None

In [6]:
from glob import glob
import os
import pandas as pd
local_base_dir = './data/'

In [7]:
subdirs = glob(os.path.join(local_base_dir, '*/'))

results = []

for subdir in subdirs:
    if 'eloise' not in subdir:
        continue
    print(f"Processing images in {subdir}")
    metadata_file_path = os.path.join(subdir, 'metadata.json')
    csv_files = glob(os.path.join(subdir, '*.csv')) c cxcv
    
    for csv_file in csv_files:
        dataset = pd.read_csv(csv_file, header=None)
        # Update here: Remove extra parentheses to correctly unpack arguments
        data_rows = [(tuple(row), metadata_file_path, local_base_dir) for index, row in dataset.iterrows()]

        for data_row in data_rows:
            # Update here: Unpack the arguments
            print(data_row)

            frame, metadata_file_path, local_base_dir = data_row
            frame = cv2.imread(frame[0])
            combined_eyes = get_combined_eyes(frame, global_sr_model, global_detector, global_predictor)

Processing images in ./data\eloise\
(('data/eloise/calibration_images/eloise_05ec3788-2e1b-41ea-9f43-12d4c7369f41.png', 545, 128, 312, 316, 291, 309, 43, 14, 412, 314, 392, 307, 42, 14, '2.94408181081119,-0.002421775822371661,0.17873982553452583', '162.9959697795879,399.84829360133364,1706.6231850433858'), './data\\eloise\\metadata.json', './data/')
(('data/eloise/calibration_images/eloise_0b8aa6d2-364c-4913-8e51-bc2ca0e5a43a.png', 370, 343, 273, 234, 260, 230, 28, 9, 339, 237, 326, 231, 28, 11, '-3.498632406740652,-0.07405489490660982,-0.0457221087864345', '-63.582490662448464,91.9290806324294,2587.4171954214466'), './data\\eloise\\metadata.json', './data/')
(('data/eloise/calibration_images/eloise_1244b4bf-0c4c-402e-a637-b709084135c3.png', 195, 665, 274, 234, 261, 230, 28, 9, 339, 237, 326, 232, 28, 10, '-3.510724246125674,-0.08051770190174923,-0.04787145210479761', '-61.713145007220106,89.24059316921195,2622.420989855196'), './data\\eloise\\metadata.json', './data/')
(('data/eloise/